In [47]:
#set up shitty model with jeopardy questions - idea being that i can replace the corpus later 
import pandas as pd #might want to use it
import numpy as np #working with arrays/matrices after all! 
import gensim # Word embeddings
import random #random choice for bot's first word
from sklearn.metrics.pairwise import cosine_similarity
import convergence_resources as cr

Thoughts: How many times does a word appear? The computer should only choose the most common words. It can't repeat any of the user_input or the bot_input.

Miles: look up most common english words in a word2vec model, use the top 20,000 words? 10,000? https://wordcounter.io/blog/how-many-words-does-the-average-person-know/   

Distance: This could be helpful: https://stackoverflow.com/questions/32446703/find-closest-vector-from-a-list-of-vectors-python

# Using GloVe

In [2]:
#using numpy 
import numpy as np
def load_glove_model(file):
    print("Loading...")
    f = open(file,'r')
    model = {}
    for line in f:
        whole_line = line.split()
        words = whole_line[0]
        embedding = np.array([float(val) for val in whole_line[1:]])
        model[words] = embedding
    print("Done.",len(model)," words loaded!")
    return model
#help from https://stackoverflow.com/questions/37793118/load-pretrained-glove-vectors-in-python

In [3]:
model = load_glove_model('glove.6B/glove.6B.300d.txt')

Loading...
Done. 400000  words loaded!


# Common English Words

Create a list of common english words from https://www.ef.edu/english-resources/english-vocabulary/top-3000-words/ with help from https://www.reddit.com/r/learnpython/comments/1zcbi4/is_there_a_way_to_get_a_list_of_common_english/

In [4]:
with open("common_words.txt") as f:
    cwordlist = []
    for line in f:
        cwordlist.append(line.strip().lower().replace('-',''))

In [5]:
common_word_vectors = {}
unvectorized_common_words = []
for word in cwordlist:
    if word in model.keys():
        common_word_vectors[word] = model[word]
    else:
        unvectorized_common_words.append(word)

In [6]:
#reshape the dictionary arrays 
common_word_vectors['truth'].shape

for key, value in common_word_vectors.items():
    common_word_vectors[key] = value.reshape(1,-1)
    
for key, value in model.items():
    model[key] = value.reshape(1,-1)

common_word_vectors['truth'].shape

(1, 300)

# Create Functions

In [52]:
#convergence function with two inputs  

def converge(user_input=str, bot_input=str):
    """Return a word that returns "average word" of the input words."""
    mean_vector = ((model[user_input] + model[bot_input])/2)
    cos_sim_dict = {}
    response_options_dict = {key: common_word_vectors[key] for key in common_word_vectors 
                             if key not in [user_input, bot_input]}
    for word, vector in response_options_dict.items():
        cos_sim_dict[float(cosine_similarity(mean_vector, vector))] = word
    max_cos_sim = max(cos_sim_dict.keys())
    bot_response = cos_sim_dict[max_cos_sim]
    return bot_response

In [35]:
# no_repeats_list = [] #tracks the answers the user and bot have already said 

# def add_inputs_to_list(user_input=str, bot_input=str):
#     """Add both inputs to the list to track potential responses."""
#     no_repeats_list.append(user_input).append(bot_input)
#     return no_repeats_list    
    

In [58]:
converge('aloha', 'goodbye')

'hello'

In [55]:
converge('hell', 'heaven')

'god'